<h1>Neteja de Dades<h1>

<h3>NYC Colissions<h3>

<h4>Vista global de la distribució de les dades i els valors NULL<h4>

In [ ]:
import pandas as pd
path = "./data/original-data.csv"

# Read the data
df = pd.read_csv(path)
print(df.head())

In [ ]:
# Convert the CRASH DATE column to datetime format
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])

# Filter the dataframe to only include rows where the CRASH DATE column is in 2019
df_2019 = df[df['CRASH DATE'].dt.year == 2019]

# Print the filtered dataframe
print(df_2019)


In [ ]:
# Show number of unique values for each column
print(df.nunique())

# Show value counts for each column
for col in df.columns:
    print(f"\nValue counts for {col}:")
    print(df[col].value_counts())


In [ ]:
# Show number of missing values for each column
print(df.isnull().sum())


<h4>Borrem Columnes no importants<h4>

In [ ]:
# Drop columns not relevant for our analysis
columns_to_drop = ['ZIP CODE', 'LATITUDE', 'LONGITUDE', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']
df2 = df.drop(columns=columns_to_drop)


<h4> Solucionem els valors NULL <h4>

In [ ]:
# Count number of missing values for each column
print(df2.isnull().sum())

In [ ]:
# Fill in missing values with 0 for the following columns:
df2['NUMBER OF PERSONS INJURED'].fillna(0, inplace=True)
df2['NUMBER OF PERSONS KILLED'].fillna(0, inplace=True)

In [ ]:
# Drop rows with missing values for the following columns:
df2.dropna(subset=['VEHICLE TYPE CODE 1'], inplace=True)


In [ ]:
# Count number of missing values in LOCATION and BOROUGH simultaneously
print((df['LOCATION'].isnull() & df['BOROUGH'].isnull()).sum())

In [ ]:
# Drop rows with missing values for both LOCATION and BOROUGH
df2.dropna(subset=['LOCATION', 'BOROUGH'], how='all', inplace=True)


In [ ]:
# Count number of missing values for each column
print(df2.isnull().sum())

In [ ]:
# Drop rows with missing values for the following columns:
df2.dropna(subset=['LOCATION'], inplace=True)
df3 = df2

<h4> Arrodonir variable CRASH TIME <h4>

In [ ]:
# Round the values in the CRASH TIME column to the nearest hour
df3['CRASH TIME'] = pd.to_datetime(df['CRASH TIME']).dt.round('H').dt.time

# Print the updated dataframe
print(df3.head())
df4 = df3

<h4>Afegir columnes interessants per a les preguntes a respondre<h4>

In [ ]:
# Create a new column that indicates whether the accident occurred on a weekday or a weekend
df4['WEEKDAY'] = pd.to_datetime(df3['CRASH DATE']).dt.dayofweek < 5

# Create a new column that indicates whether the accident occurred before or after COVID-19
covid_start_date = pd.to_datetime('2020-03-01')
df4['BEFORE COVID'] = pd.to_datetime(df3['CRASH DATE']) < covid_start_date


<h2> Responent alguna prgunta <h2>

<h5>Are there any areas with a larger number of accidents?<h5>

In [ ]:
import matplotlib.pyplot as plt

# Create a bar plot of the number of accidents by borough
df4['BOROUGH'].value_counts().plot(kind='bar')
plt.title('Number of Accidents by Borough')
plt.xlabel('Borough')
plt.ylabel('Number of Accidents')
plt.show()


In [ ]:
area_BOROUGH = {"Broklin": 179.7, "Manhattan": 58.8, "Queens":281.5, "Bronx": 109.3, "Staten Island": 148.9}

<h5>Is there any type of vehicle more prone to participate in accidents?<h5>

In [ ]:
import altair as alt

# Create a dataframe with the value counts of the 'VEHICLE TYPE CODE 1' column
df_vehicle_counts = df4['VEHICLE TYPE CODE 1'].value_counts().reset_index()
df_vehicle_counts.columns = ['Vehicle Type', 'Count']

# Create a horizontal bar chart using Altair
chart = alt.Chart(df_vehicle_counts[:10]).mark_bar().encode(
    y=alt.Y('Vehicle Type:N', sort='-x'),
    x=alt.X('Count:Q'),
    tooltip=['Vehicle Type', 'Count']
).properties(
    title='Vehicle Types Involved in Accidents'
)

chart


<h5> At what time of the day are accidents more common? <h5>

In [ ]:
# Convert the CRASH TIME column to a string
df4['CRASH TIME'] = df3['CRASH TIME'].astype(str)

# Group the data by hour and count the number of accidents for each hour
df_hourly = df4.groupby('CRASH TIME').size().reset_index(name='COUNT')

# Create a line chart of the number of accidents by hour using Altair
chart = alt.Chart(df_hourly).mark_line().encode(
    x='CRASH TIME:O',
    y='COUNT:Q'
).properties(
    title='Number of Accidents by Hour of the Day'
)

chart


<h5>¿Cuál es el promedio de personas heridas y personas muertas en los accidentes de vehículos en Nueva York? ¿Ha habido cambios significativos en estos números con el tiempo?<h5>

In [ ]:
mean_injured = df4['NUMBER OF PERSONS INJURED'].mean()
print(f"The mean number of injured people caused by car collisions is {mean_injured:.4f}")


In [ ]:
mean_killed = df4['NUMBER OF PERSONS KILLED'].mean()
print(f"The mean number of killed people caused by car collisions is {mean_killed:.4f}")


In [ ]:
# Convert the CRASH DATE column to a datetime object
df4['CRASH DATE'] = pd.to_datetime(df4['CRASH DATE'])

# Compute the yearly mean of injured people
yearly_mean_injured = df4.groupby(pd.Grouper(key='CRASH DATE', freq='A'))['NUMBER OF PERSONS INJURED'].mean().reset_index()

print(yearly_mean_injured)
# Plot the yearly mean of injured people using Altair
chart = alt.Chart(yearly_mean_injured).mark_line().encode(
    x='CRASH DATE:T',
    y='NUMBER OF PERSONS INJURED:Q'
).properties(
    title='Yearly Mean of Injured People in Car Collisions'
)

chart

In [ ]:
# Filter the dataframe to only include rows where the CRASH DATE column is in 2019
df_2019 = df4[df4['CRASH DATE'].dt.year == 2019]

# Sum the values in the NUMBER OF PERSONS INJURED column
num_injured_2019 = df_2019['NUMBER OF PERSONS INJURED'].sum()

print(f"{num_injured_2019} people got injured in 2019")


In [ ]:
# Filter the dataframe to only include rows where the CRASH DATE column is in 2019
df_2019 = df4[df4['CRASH DATE'].dt.year == 2019]

# Show all the rows from 2019
print(df_2019)
